In [21]:
!pip --quiet install bs4 langchain_chroma

In [22]:
# import library
import getpass
import os
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [23]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [24]:
loader = WebBaseLoader(web_paths=("https://python.langchain.com/v0.2/docs/concepts/",),)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [25]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [26]:
response = rag_chain.invoke({"input": "What is LangChain?"})
print(response["answer"])

LangChain is a software tool that allows users to build various applications, such as question-answering systems, chatbots, and query analysis systems, over graph databases. It provides tutorials and guides on how to use tools in a chain, add memory to chatbots, and map values to a graph database, among other functionalities. LangChain also offers integration options, API references, and templates to facilitate the development process.


In [27]:
response = rag_chain.invoke({"input": "What is a retreiver?"})
print(response["answer"])

A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store and does not need to be able to store documents, only to return them. Retrievers accept a string query as input and return a list of documents as output.


In [28]:
response = rag_chain.invoke({"input": "How do i save chat history?"})
print(response["answer"])

To save chat history, you can use the ChatHistory class in LangChain, which keeps track of inputs and outputs of the conversation. It appends these messages to a message database for future reference and interactions. Additionally, you can utilize Document objects in LangChain to store information as they contain page content and metadata, which can be loaded using Document loaders from various data sources.


In [29]:
response = rag_chain.invoke({"input": "What is the difference between using an agent and just using an llm directly?"})
print(response["answer"])

Agents are systems that use a language model (LLM) as a reasoning engine to determine actions and inputs, enabling decision-making processes based on LLM output. In contrast, using an LLM directly involves solely generating text outputs without the ability to take actions or make decisions based on those outputs. Essentially, agents add a layer of functionality on top of the LLM, allowing for more complex and dynamic interactions.


In [30]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

condense_question_system_template = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

condense_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [31]:
history_aware_retriever = create_history_aware_retriever(
    llm, vectorstore.as_retriever(), condense_question_prompt
)

hist_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = create_stuff_documents_chain(llm, hist_prompt)

hist_chain = create_retrieval_chain(history_aware_retriever, chain)

In [49]:
chathistory = []

In [50]:
human = "My name is Ayman"
response = hist_chain.invoke(
    {
        "input": human,
        "chat_history": chathistory,
    }
)
print(response["answer"])

Nice to meet you, Ayman! If you have any questions or need assistance, feel free to ask.


In [51]:
from langchain_core.messages import HumanMessage

In [53]:
chathistory.extend([HumanMessage(content=human), response["answer"]])

In [54]:
response = hist_chain.invoke(
    {
        "input": "What's my name",
        "chat_history": chathistory,
    }
)
print(response["answer"])

Your name is Ayman.


In [55]:
chathistory

[HumanMessage(content='My name is Ayman'),
 'Nice to meet you, Ayman! If you have any questions or need assistance, feel free to ask.']